Η ΣΕΙΡΑ ΑΠΑΝΤΗΣΗΣ ΕΙΝΑΙ ΕΡΩΤΗΜΑ 2, ΕΡΩΤΗΜΑ 3, ΕΡΩΤΗΜΑ 1
ΟΜΩς Η ΠΡΟΕΤΟΙΜΑΣΙΑ ΤΩΝ ΠΙΝΑΚΩΝ ΤΩΝ ΧΑΡΑΚΤΗΡΙΣΤΙΚΩΝ ΓΊΝΕΤΑΙ ΜΕ ΕΝΙΑΊΟ ΤΡΟΠΟ

ΤΟ DO για να τελειώσει η εργασία
* ΘΕΜΑ 1
Να επαναληφθει για τις άλλες 2 εταιρίες για να βρεθει η εταιρεία με την μεγαλύτερη ακρίβεια ταξινόμησης με την μέθοδο της 10πλης διεπικήρωσης (10fold validation) : το ερώτημα 1 (στο τελος του notebook) εχει k fold validation αλλα δεν καταλαβαινω αν ειναι σεταρισμένο / ενεργοποιημένο.
* ΘΕΜΑ 2
Να επαναληφθει για τις άλλες 2 εταιρίες για να βρεθει η εταιρεία με την μεγαλύτερη ακρίβεια ταξινόμησης με την μέθοδο της 10πλης διεπικήρωσης (10fold validation) ->>ΒΗΜΑ 15 https://machinelearningmastery.com/k-fold-cross-validation/ ή https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6 OK
* ΘΕΜΑ 3
Να γίνει οπως το 2 με επιπλέον χαρακτηριστικά:
τα γκολ της μιας, τα γκολ της αλλης, τα προγνωστικά για όλες τις εταιρίες OK

Gm(h) -> γκολ μιας ομαδας που επαιζε εντος εδρας

Gm(a) -> γκολ μιας ομαδας που επαιζε εκτος εδρας

Τ = {t1, t2, ..., tk} -> κ πληθους ομαδες 

Μ = {m1, m2, ..., mj} -> j πληθους αγωνες

r μια συναρτηση που το αποτελεσμα της ανηκει στο συνολο {H, D, A} -> αποτελεσμα αγωνα
r(match, home_team, away_team) = {
    return H if DGm(home_team, away_team) > 0
    return D if DGm(home_team, away_team) = 0
    return A if DGm(home_team, away_team) < 0
}

DGm(home_team, away_team) = Gm(h) - Gm(a)

R2 -> (x, y)
R8 -> (x1, x2, x3, ... x8)
Rn -> (x1, x2, x3, ..., xn)
f(t) ανηκει στο συνολο R8 και η μεταβλητη  t ειναι μια ομαδα απο το συνολο T

k ανηκει στο συνολο {Β365, BW, IW, LB} -> εταιριες

ψ(match) -> αποδοσεις για καθε πιθανο αποτελεσμα απο τις κ εταιριες (ειναι ενα τετραδα της μορφης: (a, b, c, d))
 
 
 

In [ ]:
# Δες αν υπάρχει η βάση στον φακελο


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3 as sq

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input/soccer"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

% Create a connection to the database.
% Retrieve the contents of the Team_Attributes table containing the
% following columns: id,
% Retrieve the contents of the Match table containing the following
% columns: id, B365H, B365D, B365A,BWH, BWD, BWA, IWH, IWD, IWA, LBH, LBD, LBA
and collumns needed to calculate game result and teams ids

In [ ]:
# γεμισε τους πίνακες team_att (χαρακτ ομάδας), match (προγνωστικά και αποτελέσματα)
import xml.etree.ElementTree as ET
from copy import copy



#just reading data
con = sq.connect("../input/soccer/database.sqlite")
team_att = pd.read_sql_query("SELECT * from Team_Attributes", con)
team_att = team_att [['id','team_api_id','buildUpPlaySpeed','buildUpPlayPassing','chanceCreationPassing','chanceCreationCrossing','chanceCreationShooting','defencePressure','defenceAggression','defenceTeamWidth']]
team = pd.read_sql_query("SELECT * from Team", con)
match = pd.read_sql_query("SELECT * from Match", con)
#for col in match.columns: 
#    print(col) 
# all preditind companies: 
#R28 = match[['goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession']]
#match = match[['id','match_api_id','home_team_api_id', 'home_team_goal', 'away_team_goal','away_team_api_id','B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD',
#              'LBA']]

#R28.mask(R28.astype(object).eq('None')).dropna()
##for row in R28.loc[1728]:
#    print(type(row))
#    print("\n")

show match

In [ ]:
match

#clean match from empties

In [ ]:
match.dropna()

add columns home_win, away_win, draw με 0 ή 1 με βαση ποια ομαδα εβαλε τα περισσότερα goals

In [ ]:
match.loc[(match['home_team_goal'] > match['away_team_goal']),'home_win']= int('1')
match.loc[(match['home_team_goal'] < match['away_team_goal']),'away_win']= int('1')
match.loc[(match['home_team_goal'] == match['away_team_goal']),'draw']= int('1')
match

In [ ]:
# replace NaN with 0
match.fillna(0, inplace=True)
match

Mετατροπη των αποδόσεων σε προτεινόμενες πιθανότητες (implied probabilities). Έτσι έχουμε εύρος απο 0 εως 1 για όλα τα χαρακτηριστικά.

In [ ]:
# κανω drop τις σειρες που εχουν odds 0.00

#match[(match.T !=0).any()]
match = match[match.B365H != 0]
match = match[match.BWH != 0]
match = match[match.IWD != 0]
match = match[match.LBH != 0]
# match
match2=match

match2['B365H'] = 1/match['B365H']
match2['B365D'] = 1/match['B365D']
match2['B365A'] = 1/match['B365A']
match2['BWH'] = 1/match['BWH']
match2['BWD'] = 1/match['BWD']
match2['BWA'] = 1/match['BWA']
match2['IWH'] = 1/match['IWH']
match2['IWD'] = 1/match['IWD']
match2['IWA'] = 1/match['IWA']
match2['LBH'] = 1/match['LBH']
match2['LBD'] = 1/match['LBD']
match2['LBA'] = 1/match['LBA']
match2



In [ ]:
#match2.replace([np.inf, -np.inf], np.nan)
#match2.dropna

Ο πίνακας team_att περιέχει τα 8 χαρακτηριστικά κάθε ομάδας, όπως χρειάζονται για το ερώτημα 3

In [ ]:
team_att

In [ ]:
# Για το ερώτημα 3 φτιάχνω ενα join table απο τα match και team_att με βαση home_team_api_id	away_team_api_id, επίσης κρατάω μόνο τα προγνωστικά για τις 4 εταιρίες
# ενδιαμεσοι πίνακες για καθε ομαδα με τα χαρακτηριστικά τους
team_att.rename(columns = {'team_api_id':'home_team_api_id'}, inplace = True)
#away_team_att = team_att.rename(columns = {'home_team_api_id':'away_team_api_id'}, inplace = True)



Κάνω JOIN τους πίνακες team_att και match με βάση το κλειδι team_api_id = home_team_api_id, και επίσης για το team_api_id = away_team_api_id

In [ ]:
match2

In [ ]:
team_att

Βλέπουμε οτι επαναλαμβάνονται οι εμφανίσεις των ίδιων χαρακτηριστικών (εξελίσονται στον χρόνο) οπότε επιλέγουμε να χρησιμοποιήσουμε το mean των εκάστοτε χαρακτηριστικών με για κάθε ομάδα

In [ ]:
team_att2=team_att.groupby(['home_team_api_id']).agg({'buildUpPlaySpeed':'mean','buildUpPlayPassing':'mean','chanceCreationPassing':'mean','chanceCreationCrossing':'mean','chanceCreationShooting':'mean','defencePressure':'mean','defenceAggression':'mean','defenceTeamWidth':'mean'})
#team_att2=team_att2.groupby(['home_team_api_id']).agg({'buildUpPlaySpeed':'mean','buildUpPlayPassing':'mean','chanceCreationPassing':'mean','chanceCreationCrossing':'mean','chanceCreationShooting':'mean','defencePressure':'mean','defenceAggression':'mean','defenceTeamWidth':'mean'})

In [ ]:
team_att

..και κανουμε merge τα χαρακτηριστικά με τα odds

In [ ]:
q3 = pd.merge(match2, team_att2, on=['home_team_api_id'])
#q3 = pd.merge(match2, away_team_att, on=['away_team_api_id'])

In [ ]:
q3 = pd.merge(match2, team_att2, on=['home_team_api_id'])


In [ ]:
q3= q3.rename(columns={'buildUpPlaySpeed': 'home_buildUpPlaySpeed', 'buildUpPlayPassing': 'home_buildUpPlayPassing','chanceCreationPassing': 'home_chanceCreationPassing', 'chanceCreationCrossing': 'home_chanceCreationCrossing','chanceCreationShooting': 'home_chanceCreationShooting', 'defencePressure': 'home_defencePressure','defenceAggression': 'home_defenceAggression', 'defenceTeamWidth': 'home_defenceTeamWidth'})

In [ ]:
q3 # O Πίνακας με τα χαρακτηρισιτκά εισαγωγής για το ερώτημα 3 (TNN με 28 χαρακτ)

In [ ]:
team_att

Εισαγω στον γενικό πίνακα τα χαρακτηριστικά της home team ( με join )

In [ ]:
team_att.rename(columns = {'team_api_id':'away_team_api_id'}, inplace = True)


In [ ]:
team_att=team_att.groupby(['home_team_api_id']).agg({'buildUpPlaySpeed':'mean','buildUpPlayPassing':'mean','chanceCreationPassing':'mean','chanceCreationCrossing':'mean','chanceCreationShooting':'mean','defencePressure':'mean','defenceAggression':'mean','defenceTeamWidth':'mean'})

In [ ]:
team_att.index.names = ['away_team_api_id']

In [ ]:
team_att

In [ ]:
team_att = team_att.rename(columns={'buildUpPlaySpeed': 'away_buildUpPlaySpeed', 'buildUpPlayPassing': 'away_buildUpPlayPassing','chanceCreationPassing': 'away_chanceCreationPassing', 'chanceCreationCrossing': 'away_chanceCreationCrossing','chanceCreationShooting': 'away_chanceCreationShooting', 'defencePressure': 'away_defencePressure','defenceAggression': 'away_defenceAggression', 'defenceTeamWidth': 'away_defenceTeamWidth'})
#team_att.rename(columns = {'home_team_api_id':'away_team_api_id'}, inplace = False)

In [ ]:
team_att

In [ ]:
q3 = pd.merge(match2, team_att, on=['away_team_api_id'])

In [ ]:
team_att2

In [ ]:
q3 = pd.merge(match2, team_att2, on=['home_team_api_id'])
q3 = pd.merge(q3, team_att, on=['away_team_api_id'])

In [ ]:
q3.loc[(q3['home_win'] == 1.0),'result'] = 1 #swsto
q3.loc[(q3['away_win'] == 1.0),'result'] = 2
q3.loc[(q3['draw'] == 1.0),'result'] = 3

In [ ]:
q3 = q3.dropna() #swsto

q3

> O πίνακας  deep_attrib έχει μόνο τις στήλες που χρειαζόμαστε, υποσύνολο του q3

In [ ]:
deep_attrib = q3[['B365H','B365D','B365A','BWH','BWD','BWA','IWH','IWD','IWA','LBH','LBD','LBA','buildUpPlaySpeed','buildUpPlayPassing','chanceCreationPassing','chanceCreationCrossing','chanceCreationShooting','defencePressure','defenceAggression','defenceTeamWidth','away_buildUpPlaySpeed','away_buildUpPlayPassing','away_chanceCreationPassing','away_chanceCreationCrossing','away_chanceCreationShooting','away_defencePressure','away_defenceAggression','away_defenceTeamWidth','result']].copy()

In [ ]:
deep_attrib

In [ ]:
team_att

In [ ]:
# γραμμικο νευρωνικο 


In [ ]:
#shuffle match rows so split tables are randomized
#match = match.reindex(np.random.permutation(match.index))

#split match data into training, validation, and test sets
m_train = match.iloc[:17861]
m_valid = match.iloc[17861:21108]
m_test = match.iloc[21108:]

--ΑΛΛΑΓΗ ΤΟ ΠΑΡΑΚΑΤΩ ΣΕ ΕΛΛΗΝΙΚΑ---

We take one company data only + classes, put it in X array
We need to grab the data (the information on each sample) from the pandas array and put it into a nice numpy one.

X=match[['column1','column2','column3'....]]
X = np.array(X)
X[:5]

classes are already hot encoded (1,0,0).. for home win.. 
we have to put classes in Y



In [ ]:
X=match2[['B365H','B365D','B365A']]
X = np.array(X)
Y=match2[['home_win','away_win','draw']]
Y=np.array(Y)
X[:5]

In [ ]:
Y[:5]

In [ ]:
X

In [ ]:
X

Next we will split our dataset into train/validation/test using sklearn. Initially the data will be split into train/test and then the training data will be further split into train/validation.

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1)

we are going to build a simple neural network that supports multiple layers and validation. The main function is NeuralNetwork, which will train the network for the specified number of epochs. At first, the weights of the network will get randomly initialized by InitializeWeights. Then, in each epoch, the weights will be updated by Train and finally, every 20 epochs accuracy both for the training and validation sets will be printed by the Accuracy function. As input the function receives the following:

X_train, Y_train: The training data and target values.
X_val, Y_val: The validation data and target values. These are optional parameters.
epochs: Number of epochs. Defaults at 10.
nodes: A list of integers. Each integer denotes the number of nodes in each layer. The length of this list denotes the number of layers. That is, each integer in this list corresponds to the number of nodes in each layer.
lr: The learning rate of the back-propagation training algorithm. Defaults at 0.15.

def NeuralNetwork(X_train, Y_train, X_val=None, Y_val=None, epochs=2, nodes=[], lr=0.15):
    hidden_layers = len(nodes) - 1
    weights = InitializeWeights(nodes)

    for epoch in range(1, epochs+1):
        weights = Train(X_train, Y_train, lr, weights)
        
        print("epoch = {}".format(epoch))
        print("Training Accuracy:{}".format(Accuracy(X_train, Y_train, weights)))
        if X_val.any():
                print("Validation Accuracy:{}".format(Accuracy(X_val, Y_val, weights)))
        
        #if(epoch % 20 == 0):
        #    print("Epoch {}".format(epoch))
        #    print("Training Accuracy:{}".format(Accuracy(X_train, Y_train, weights)))
        #    if X_val.any():
        #        print("Validation Accuracy:{}".format(Accuracy(X_val, Y_val, weights)))
            
    return weights

The weights of the network are initialized randomly in the range [-1, 1] by InitializeWeights. This function takes as input nodes and returns a multi-dimensional array, weights. Each element in the weights list represents a hidden layer and holds the weights of connections from the previous layer (including the bias) to the current layer. So, element i in weights holds the weights of the connections from layer i-1 to layer i. Note that the input layer has no incoming connections so it is not present in weights.

For example, let's say we have four features (as is the case with the Iris dataset) and the hidden layers have 5, 10 and 3 (for the output, one for each class) nodes. Thus, nodes == [4, 5, 10, 3] Then, the connections between the input layer and the first hidden layer will be (4+1)*5 = 25. After augmenting the input with the bias (in this case the bias has a constant value of 1), the input layer has 5 nodes. By fully connecting this layer to the next (each node in the input layer is connected will every node of the hidden layer), we get that the total number of connections is 25. Similarly, we get that the connections between the first hidden layer and the second one will be (5+1)*10 = 60 and between the second hidden layer with the output we have (10+1)*3 = 33 connections.

In the implementation, numpy is used to generate a random number in the [-1, 1] range for each connection.

def InitializeWeights(nodes):
    """Initialize weights with random values in [-1, 1] (including bias)"""
    layers, weights = len(nodes), []
    
    for i in range(1, layers):
        w = [[np.random.uniform(-1, 1) for k in range(nodes[i-1] + 1)]
              for j in range(nodes[i])]
        weights.append(np.matrix(w))
    
    return weights

With the weights of the network at hand, we want to continuously adjust them across the epochs so that (hopefully) our network becomes more accurate. The training of the weights is accomplished via the popular (Forward) Back-Propagation algorithm. In this technique, the input first passes through the whole network and the output is calculated. Then, according to the error of this output, the weights of the network are updated from last to first. The error is propagated backwards, hence the name of the titular algorithm. Let's get into more detail about these two steps:

Forward Propagation:

Each layer receives an input and computes an output. The output is computed by first calculating the dot product between the input and the weights of the layer and then passing this dot product through an activation function (in this case, the sigmoid function).
The output of each layer is the input of the next.
The input of the first layer is the feature vector.
The output of the final layer is the prediction of the network.

def ForwardPropagation(x, weights, layers):
    activations, layer_input = [x], x
    for j in range(layers):
        activation = Sigmoid(np.dot(layer_input, weights[j].T))
        activations.append(activation)
        layer_input = np.append(1, activation) # Augment with bias
    
    return activations

def BackPropagation(y, activations, weights, layers):
    outputFinal = activations[-1]
    error = np.matrix(y - outputFinal) # Error at output
    
    for j in range(layers, 0, -1):
        currActivation = activations[j]
        
        if(j > 1):
            # Augment previous activation
            prevActivation = np.append(1, activations[j-1])
        else:
            # First hidden layer, prevActivation is input (without bias)
            prevActivation = activations[0]
        
        delta = np.multiply(error, SigmoidDerivative(currActivation))
        weights[j-1] += lr * np.multiply(delta.T, prevActivation)

        w = np.delete(weights[j-1], [0], axis=1) # Remove bias from weights
        error = np.dot(delta, w) # Calculate error for current layer
    
    return weights

    def Train(X, Y, lr, weights):
        layers = len(weights)
        for i in range(len(X)):
            x, y = X[i], Y[i]
            x = np.matrix(np.append(1, x)) # Augment feature vector
        
            activations = ForwardPropagation(x, weights, layers)
            weights = BackPropagation(y, activations, weights, layers)

        return weights

def Sigmoid(x):
    return 1 / (1 + np.exp(-x))

def SigmoidDerivative(x):
    return np.multiply(x, 1-x)

def Predict(item, weights):
    layers = len(weights)
    item = np.append(1, item) # Augment feature vector
    
    ##_Forward Propagation_##
    activations = ForwardPropagation(item, weights, layers)
    
    outputFinal = activations[-1].A1
    index = FindMaxActivation(outputFinal)

    # Initialize prediction vector to zeros
    y = [0 for i in range(len(outputFinal))]
    y[index] = 1  # Set guessed class to 1

    return y # Return prediction vector


def FindMaxActivation(output):
    """Find max activation in output"""
    m, index = output[0], 0
    for i in range(1, len(output)):
        if(output[i] > m):
            m, index = output[i], i
    
    return index

def Accuracy(X, Y, weights):
    """Run set through network, find overall accuracy"""
    correct = 0

    for i in range(len(X)):
        x, y = X[i], list(Y[i])
        guess = Predict(x, weights)

        if(y == guess):
            # Guessed correctly
            correct += 1

    return correct / len(X)

f = len(X[0]) # Number of features
o = len(Y[0]) # Number of outputs / classes

layers = [f, 5, 10, o] # Number of nodes in layers
lr, epochs = 0.15, 10 #100

weights = NeuralNetwork(X_train, Y_train, X_val, Y_val, epochs=epochs, nodes=layers, lr=lr);

print("Testing Accuracy: {}".format(Accuracy(X_test, Y_test, weights)))

Testing Accuracy: 0.5143874221299318 me 20 epochs 0.15 step

Enalaktikos kwdikos gia deep nn me k-fold validation (ΕΡΩΤΗΜΑ 2)

In [ ]:
X

ΦΤΙΑΧΝΩ 4 ΠΙΝΑΚΕΣ ΠΟΥ ΠΕΡΙΈΧΟΥΝ ΤΑ ΠΡΟΓΝΩΣΤΙΚΆ ΚΑΙ ΤΑ ΑΠΟΤΕΛΕΣΜΑΤΑ - ΕΝΑΝ ΠΙΝΑΚΑ ΓΙΑ ΚΑΘΕ ΕΤΑΙΡΙΑ

In [ ]:
import pandas as pd
import numpy as np
# companies = 'B365H','B365D','B365A', 'BWH','BWD','BWA', 'IWH','IWD','IWA', 'LBH','LBD','LBA'
pd.options.mode.chained_assignment = None  # default='warn'
matchB365 = match[["B365H","B365A","B365D","home_team_goal","away_team_goal"]]
matchB365["result"]= np.nan

matchBW =  match[['BWH','BWD','BWA',"home_team_goal","away_team_goal"]]
matchBW["result"]= np.nan

matchIW =  match[['IWH','IWD','IWA',"home_team_goal","away_team_goal"]]
matchIW["result"]= np.nan

matchLB =  match[['LBH','LBD','LBA',"home_team_goal","away_team_goal"]]
matchLB["result"]= np.nan

#matchB365.loc[:, 'result'] = 0
matchB365

In [ ]:
matchB365.loc[(matchB365['home_team_goal'] > matchB365['away_team_goal']),'result']= int('1')
matchB365.loc[(matchB365['home_team_goal'] < matchB365['away_team_goal']),'result']= int('2')
matchB365.loc[(matchB365['home_team_goal'] == matchB365['away_team_goal']),'result']= int('3')
#matchB365['result'] = np.where(matchB365['home_win']== '1.0', True, False)
matchB365 = matchB365[["B365H","B365A","B365D","result"]]

matchBW["result"] = matchB365["result"].values
matchIW["result"]= matchB365["result"].values
matchLB["result"]= matchB365["result"].values

matchB365

In [ ]:
matchIW

In [ ]:

#matchB365.loc[(matchB365['home_team_goal'] = '1.0'),'result']= int('1')

In [ ]:
#matchB365["result"] = np.where((matchB365.home_win="1.0"),true,false)

In [ ]:
matchBW

In [ ]:
matchB365=matchB365.to_numpy()
matchBW=matchBW.to_numpy()
matchIW=matchIW.to_numpy()
matchLB=matchLB.to_numpy()

In [ ]:
matchB365

In [ ]:
Y=matchB365[:,3]
YBW=matchBW[:,3]
YIW=matchIW[:,3]
YLB=matchLB[:,3]

Y.shape

ΧΩΡΙΖΩ ΤΟΝ ΚΑΘΕ ΠΙΝΑΚΑ ΣΕ ΠΡΟΓΝΩΣΤΙΚΑ ΚΑΙ ΑΠΟΤΕΛΕΣΜΑ για τα Ερωτήματα 1 και 2

In [ ]:
X=matchB365[:,0:2]
Y=matchB365[:,3]

XBW=matchBW[:,0:2]
YBW=matchBW[:,5]

XIW=matchIW[:,0:2]
YIW=matchIW[:,5]

XLB=matchLB[:,0:2]
YLB=matchLB[:,5]

In [ ]:
#X=X.to_numpy()
#Y=Y.to_numpy()

# Ερώτημα 2
# Αρχίζουμε την εκπαίδευση του πολυστρωματικού νευρωνικού δικτύου (3 χαρακτηριστικά / εταιρεία)

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier

#f = open("seeds_dataset.txt")
#data = np.loadtxt(f)


kf = KFold(n_splits=10)
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3, 28), random_state=1,max_iter=900) #hidden_layer_sizes=(samples=results ,features)

'buildUpPlaySpeed','buildUpPlayPassing','chanceCreationPassing','chanceCreationCrossing','chanceCreationShooting','defencePressure','defenceAggression','defenceTeamWidth','away_buildUpPlaySpeed','away_buildUpPlayPassing','away_chanceCreationPassing','away_chanceCreationCrossing','away_chanceCreationShooting','away_defencePressure','away_defenceAggression','away_defenceTeamWidth'

meanB365 = 0
for train_indices, test_indices in kf.split(X):
    clf.fit(X[train_indices], Y[train_indices])
    print(clf.predict(X[test_indices]))
    current_score = clf.score(X[test_indices], Y[test_indices])
    print(current_score)
    meanB365 += current_score
meanB365 = meanB365 / 10
print("average score for B365 is: {}".format(meanB365))
print("")

kf2 = KFold(n_splits=10)
clf2 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3, 28), random_state=1,max_iter=900) #hidden_layer_sizes=(samples=results ,features)

'buildUpPlaySpeed','buildUpPlayPassing','chanceCreationPassing','chanceCreationCrossing','chanceCreationShooting','defencePressure','defenceAggression','defenceTeamWidth','away_buildUpPlaySpeed','away_buildUpPlayPassing','away_chanceCreationPassing','away_chanceCreationCrossing','away_chanceCreationShooting','away_defencePressure','away_defenceAggression','away_defenceTeamWidth'

meanBW = 0
for train_indices, test_indices in kf2.split(XBW):
    clf2.fit(XBW[train_indices], YBW[train_indices])
    print(clf2.predict(XBW[test_indices]))
    current_score = clf2.score(XBW[test_indices], YBW[test_indices])
    print(current_score)
    meanBW += current_score
meanBW = meanBW / 10
print("average score for BW is: {}".format(meanBW))
print("")

kf3 = KFold(n_splits=10)
clf3 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3, 28), random_state=1,max_iter=900) #hidden_layer_sizes=(samples=results ,features)

'buildUpPlaySpeed','buildUpPlayPassing','chanceCreationPassing','chanceCreationCrossing','chanceCreationShooting','defencePressure','defenceAggression','defenceTeamWidth','away_buildUpPlaySpeed','away_buildUpPlayPassing','away_chanceCreationPassing','away_chanceCreationCrossing','away_chanceCreationShooting','away_defencePressure','away_defenceAggression','away_defenceTeamWidth'

meanLW = 0
for train_indices, test_indices in kf3.split(XIW):
    clf3.fit(XIW[train_indices], YIW[train_indices])
    print(clf3.predict(XIW[test_indices]))
    current_score = clf3.score(XIW[test_indices], YIW[test_indices])
    print(current_score)
    meanLW += current_score
meanLW = meanLW / 10
print("average score for LW is: {}".format(meanLW))
print("")

kf4 = KFold(n_splits=10)
clf4 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3, 28), random_state=1,max_iter=900) #hidden_layer_sizes=(samples=results ,features)

'buildUpPlaySpeed','buildUpPlayPassing','chanceCreationPassing','chanceCreationCrossing','chanceCreationShooting','defencePressure','defenceAggression','defenceTeamWidth','away_buildUpPlaySpeed','away_buildUpPlayPassing','away_chanceCreationPassing','away_chanceCreationCrossing','away_chanceCreationShooting','away_defencePressure','away_defenceAggression','away_defenceTeamWidth'

meanLB = 0
for train_indices, test_indices in kf4.split(XLB):
    clf4.fit(XLB[train_indices], YLB[train_indices])
    print(clf4.predict(XLB[test_indices]))
    current_score = clf4.score(XLB[test_indices], YLB[test_indices])
    print(current_score)
    meanLB += current_score
meanLB = meanLB / 10
print("average score for LB is: {}".format(meanLB))
print("")

meanDict = {meanB365:"B365", meanBW:"BW", meanLW:"LW", meanLB:"LB"}
company_mean = max([meanB365, meanBW, meanLW, meanLB])

In [ ]:
print("max average is for company : {}".format(meanDict[company_mean]))
meanDict

# ΕΡΩΤΗΜΑ 3

(Εχουμε φτιαξει απο πριν τον πίνακα deep_attrib με τα απαραίτητα χαρακτηριστικά ν=28 που είναι τα odds απο 4 εταιρίες, και τα 8 χαρακτηριστικά απο κάθε ομάδα)

In [ ]:
X3 = deep_attrib.to_numpy()
X3= X3[:,0:28]


In [ ]:
X3

In [ ]:
Y3 = deep_attrib.to_numpy()
Y3 = Y3[:,28]

In [ ]:
Y3

Φτιαχνουμε ενα δίκτυο Multi-layer Perceptron

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier

#f = open("seeds_dataset.txt")
#data = np.loadtxt(f)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X3 = sc.fit_transform(X3)

#https://scikit-learn.org/stable/modules/grid_search.html

kf2 = KFold(n_splits=10)
clf2 = MLPClassifier(solver='adam', activation ='relu', alpha=1e-5, hidden_layer_sizes=(5, 28), random_state=1,max_iter=1000,learning_rate_init=.1) # hidden_layer_sizes=(5, 28),
for train_indices, test_indices in kf2.split(X3):
    clf2.fit(X3[train_indices], Y3[train_indices])
   # print(clf2.predict_proba(X3[test_indices]))
    print(clf2.score(X3[test_indices], Y3[test_indices]))
    print(clf2.predict(X3[train_indices]))
    print(clf2.predict(X3[test_indices]))
    

diagrams and attributed for trainning https://www.python-course.eu/neural_networks_with_scikit.php

https://www.kaggle.com/ahmethamzaemra/mlpclassifier-example

-------------------------------------------------------------------

# **Ερωτημα 1**

Γραμμικο μονοστρωματικο δικτυο. προσωρινα κανω export το dataframe σε csv. Ο κωδικάς του δεν θελει one hot για τα labels αλλα ονομα, οπότε πρεπει να φτιάξω στηλη με label-classes: home/away/draw αναλογα μετα αποτελεσματα.


In [ ]:
# ERWTIMA 1
match2

In [ ]:
# ERWTIMA 1
match.loc[(match['home_win'] == 1),'result']= 'home_win'
match.loc[(match['away_win'] == 1),'result']= 'away_win'
match.loc[(match['draw'] == 1),'result']= 'draw'
match.loc[(match['home_team_goal'] > match['away_team_goal']),'home_win']= int('1')
match.loc[(match['home_team_goal'] < match['away_team_goal']),'away_win']= int('1')
match.loc[(match['home_team_goal'] == match['away_team_goal']),'draw']= int('1')
match = match2[match2.result != 0]

perceptronB365 = match[['B365H', 'B365D', 'B365A','result']]
perceptronB365 = perceptronB365[perceptronB365.result != 0]
perceptronB365 = perceptronB365[perceptronB365.B365H != 0]
perceptronB365 = perceptronB365[perceptronB365.B365D != 0]
perceptronB365 = perceptronB365[perceptronB365.B365A != 0]

perceptronIW = match[['IWH', 'IWD', 'IWA','result']]
perceptronIW = perceptronIW[perceptronIW.result != 0]
perceptronIW = perceptronIW[perceptronIW.IWH != 0]
perceptronIW = perceptronIW[perceptronIW.IWD != 0]
perceptronIW = perceptronIW[perceptronIW.IWA != 0]

perceptronLB = match[['LBH', 'LBD', 'LBA','result']]
perceptronLB = perceptronLB[perceptronLB.result != 0]
perceptronLB = perceptronLB[perceptronLB.LBH != 0]
perceptronLB = perceptronLB[perceptronLB.LBD != 0]
perceptronLB = perceptronLB[perceptronLB.LBA != 0]

perceptronBW = match[['BWH', 'BWD', 'BWA','result']]
perceptronBW = perceptronBW[perceptronBW.result != 0]
perceptronBW = perceptronBW[perceptronBW.BWH != 0]
perceptronBW = perceptronBW[perceptronBW.BWD != 0]
perceptronBW = perceptronBW[perceptronBW.BWA != 0]

perceptronB365.fillna('home_win', inplace=True)
perceptronB365.to_csv(r'soccer_data_perceptronB365.csv', index = False)

perceptronIW.fillna('home_win', inplace=True)
perceptronIW.to_csv(r'soccer_data_perceptronIW.csv', index = False)

perceptronLB.fillna('home_win', inplace=True)
perceptronLB.to_csv(r'soccer_data_perceptronLB.csv', index = False)

perceptronBW.fillna('home_win', inplace=True)
perceptronBW.to_csv(r'soccer_data_perceptronBW.csv', index = False)

In [ ]:
# ERWTIMA 1
from subprocess import check_output
print(check_output(["ls", "/kaggle/working"]).decode("utf8"))
#file = open(“soccer_data_perceptron.csv”,”r”) 
#print file.read() 
perceptronIW

In [ ]:
# ERWTIMA 1

import math; #For pow and sqrt
from random import shuffle;


###_Read Data_###
def ReadData(fileName):
    #Read the file, splitting by lines
    f = open(fileName,'r');
    lines = f.read().splitlines();
    f.close();

    #Split the first line by commas, remove the last element
    #and save the length of the rest.
    featuresNumber = len(lines[0].split(','));

    items = [];
    classes = [];
    features = lines[0].split(',')[:-1];
    print(features)

    for i in range(1, len(lines)):
        line = lines[i].split(',');

        if(line[-1] not in classes):
            classes.append(line[-1]);

        itemFeatures = {"Class" : line[-1], "Bias" : 1};

        for j in range(len(features)):
            f = features[j]; #Get the feature at index j
            v = float(line[j]);

            itemFeatures[f] = v;
    
        items.append(itemFeatures);

    shuffle(items);
    
    print(classes)

    return items,classes,features;


###_Evaluation Functions_###
def K_FoldValidation(K, Items, rate, epochs, classes, features):
    if(K > len(Items)):
        return -1;

    correct = 0; #The number of correct classifications
    total = len(Items)*(K-1); #The total number of classifications

    l = int(len(Items)/K); #The length of a fold

    for i in range(K):
        #Split data set into training and testing
        trainingSet = Items[i*l:(i+1)*l];
        testSet = Items[:i*l] + Items[(i+1)*l:];

        weights = CalculateWeights(trainingSet, rate, epochs, classes, features);

        for item in testSet:
            itemClass = item["Class"];

            itemFeatures = {};

            for key in item:
                if(key != "Class"):
                    #If key isn't "Class", add it to itemFeatures
                    itemFeatures[key] = item[key];
          
            guess = Perceptron(itemFeatures, weights);

            if(guess == itemClass):
                #Guessed correctly
                correct += 1;

    return correct/float(total);

def Evaluate(times, K, Items, rate, epochs, classes, features):
    accuracy = 0;
    for t in range(times):
        shuffle(Items);
        accuracy += K_FoldValidation(K, Items, rate, epochs, classes, features);
    print(accuracy);
    print((accuracy)/float(times)); # kai to % nomizw gia to upoloipo mias diairesis


###_Auxiliary Functions_###
def AddDictionaries(d1, d2, rate):
    d3 = {};
    for i in d1:
        d3[i] = d1[i] + rate*d2[i];

    return d3;

def SubDictionaries(d1, d2, rate):
    d3 = {};
    for i in d1:
        d3[i] = d1[i] - rate*d2[i];

    return d3;

def CalculateConfidence(item, weight):
    #Add the product of the weight and item values for each feature
    confidence = 0;

    for k in weight:
        confidence += weight[k]*item[k];

    return confidence;


###_Core Functions_###
def CalculateWeights(trainingSet, rate, epochs, classes, features):
    #Initialize weights at 0
    weights = {};

    #Initialize weights dictionary. Weights is divided in classes.
    #Each class has its own dictionary, which is numerical values/weights
    #for the features.
    for c in classes:
        weights[c] = {"Bias":0};
        for f in features:
            weights[c][f] = 0;

    for epoch in range(epochs):
        for item in trainingSet:
            #Iterate through trainingSet
            #Guess where item belongs
            y = -1;
            guess = "";
            for w in weights:
                confidence = CalculateConfidence(item, weights[w]);

                if(confidence > y):
                    y = confidence;
                    guess = w;

            correct = item["Class"];
            if(correct != guess):
                weights[guess] = SubDictionaries(weights[guess], item, rate);
                weights[correct] = AddDictionaries(weights[correct], item, rate);
                
    return weights;

def Perceptron(item, weights):
    item["Bias"] = 1; #Augment item vector with bias
    m = -1; #Hold the maximum
    classification = "";

    #Calculate chance of item being in each class,
    #pick the maximum.
    for w in weights:
        #Multiply the item vector with the class weights vector
        guess = CalculateConfidence(item, weights[w]);
        if(guess > m):
            #Our guess is better than our current best guess,
            #update max and classification
            m = guess;
            classification = w;

    return classification;


itemsB365, classesB365, featuresB365 = ReadData('/kaggle/working/soccer_data_perceptronB365.csv');
itemsBW, classesBW, featuresBW = ReadData('/kaggle/working/soccer_data_perceptronBW.csv');
itemsIW, classesIW, featuresIW = ReadData('/kaggle/working/soccer_data_perceptronIW.csv');
itemsLB, classesLB, featuresLB = ReadData('/kaggle/working/soccer_data_perceptronLB.csv');
    
lRate = 0.1;
epochs = 10;

    

In [ ]:
print("Τα αποτελέσματα χρησιμοποιόντας τα προγνωστικά της IW:")
weightsIW = CalculateWeights(itemsIW, lRate, epochs, classesIW, featuresIW);
Evaluate(100, 10, itemsIW, lRate, epochs, classesIW, featuresIW);

Το αποτέλεσμα του παραπάνω

['IWH', 'IWD', 'IWA']

['draw', 'away_win', 'home_win']

41.93545100715618

0.4193545100715618

**με 100 επαναλήψεις**

42.093742427164784

0.4209374242716478

Τα αποτελέσματα χρησιμοποιόντας τα προγνωστικά της Bwin:

In [ ]:
weightsBW = CalculateWeights(itemsBW, lRate, epochs, classesBW, featuresBW);
Evaluate(100, 10, itemsBW, lRate, epochs, classesBW, featuresBW);


Τα αποτελέσματα χρησιμοποιόντας τα προγνωστικά της B365:

In [ ]:
weightsB365 = CalculateWeights(itemsB365, lRate, epochs, classesB365, featuresB365);
Evaluate(100, 10, itemsB365, lRate, epochs, classesB365, featuresB365);


Τα αποτελέσματα χρησιμοποιόντας τα προγνωστικά της LB:

In [ ]:
weightsLB = CalculateWeights(itemsLB, lRate, epochs, classesLB, featuresLB);
Evaluate(100, 10, itemsLB, lRate, epochs, classesLB, featuresLB);

ΤΕΛΟΣ απαντήσεων για τα ερωτήματα 1,2,3